In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# parameters setting and load data

In [2]:
max_steps = 1000
learning_rate = 0.001
dropout = 0.9
data_dir='MNIST_data'
log_dir='logs/mnist_with_summaries'

mnist = input_data.read_data_sets(data_dir, one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


# image summary to tensorboard

In [3]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

# nn model init function

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Variable summary function

In [5]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

# mlp nn summary function

In [6]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
        return activations

# dropout summary

In [7]:
hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)
    
y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

# loss, optimizer, accuracy and summary.scalar

In [8]:
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

# summary merge and file writer

In [9]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')
tf.global_variables_initializer().run()

# feed_dict dropout function

In [10]:
def feed_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(100)
        k = dropout
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

# train, test and log save

In [11]:
saver = tf.train.Saver()
for i in range(max_steps):
    if i % 10 == 0:
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:
        if i % 100 == 99:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True),
                                 options=run_options, run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            saver.save(sess, log_dir+"/model.ckpt", i)
            print("Adding run metadata for", i)
        else:
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
            train_writer.add_summary(summary, i)
train_writer.close()
test_writer.close()

Accuracy at step 0: 0.0733
Accuracy at step 10: 0.6929
Accuracy at step 20: 0.8253
Accuracy at step 30: 0.862
Accuracy at step 40: 0.8807
Accuracy at step 50: 0.8926
Accuracy at step 60: 0.9033
Accuracy at step 70: 0.9052
Accuracy at step 80: 0.9109
Accuracy at step 90: 0.9089
Adding run metadata for 99
Accuracy at step 100: 0.9177
Accuracy at step 110: 0.9209
Accuracy at step 120: 0.9233
Accuracy at step 130: 0.927
Accuracy at step 140: 0.9236
Accuracy at step 150: 0.9254
Accuracy at step 160: 0.9305
Accuracy at step 170: 0.9285
Accuracy at step 180: 0.9295
Accuracy at step 190: 0.9321
Adding run metadata for 199
Accuracy at step 200: 0.9337
Accuracy at step 210: 0.9341
Accuracy at step 220: 0.9372
Accuracy at step 230: 0.9373
Accuracy at step 240: 0.937
Accuracy at step 250: 0.9422
Accuracy at step 260: 0.942
Accuracy at step 270: 0.9415
Accuracy at step 280: 0.942
Accuracy at step 290: 0.9403
Adding run metadata for 299
Accuracy at step 300: 0.9424
Accuracy at step 310: 0.9455
Accur

# command on console:

## tensorboard --logdir=logs/mnist_with_summaries